In [1]:
import json
import numpy as np

In [2]:
with open('../data/gold_set_inherit.data', "r") as f:
    gold_set = f.read().split('\n')[:-1]
f.close()

In [3]:
query_len = 2
query_per_set = 10
query_list = []
for st in gold_set:
    json_st = json.loads(st)
    count = 0
    for i in range(query_per_set):
        q = {}
        q['title'] = json_st['title'].lower()
        q['qid'] = '{}|{}|{}'.format(json_st['id'], query_len, count)
        q['entities'] = np.random.choice(json_st['entities'], query_len, replace=False).tolist()
        query_list.append(json.dumps(q))
        count += 1

In [4]:
with open('../data/query-2.txt', "w+") as f:
    f.write('\n'.join(query_list))
f.close()